In [135]:
import pandas as pd
import numpy as np

In [136]:
df = pd.read_csv('../../../data_set/train-data.csv', index_col=0)

Prima di tutto osserviamo la panoramica generale del dataset

In [137]:
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,NaN,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,NaN,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,8.61 Lakh,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,NaN,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,NaN,17.74


Già notiamo la mancanza di uniformità nelle unità di misura della feature Mileage, probabilmente dovuta al carburante differente. Per comprendere a pieno la natura delle features:

In [139]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6019 entries, 0 to 6018
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               6019 non-null   object 
 1   Location           6019 non-null   object 
 2   Year               6019 non-null   int64  
 3   Kilometers_Driven  6019 non-null   int64  
 4   Fuel_Type          6019 non-null   object 
 5   Transmission       6019 non-null   object 
 6   Owner_Type         6019 non-null   object 
 7   Mileage            6017 non-null   object 
 8   Engine             5983 non-null   object 
 9   Power              5983 non-null   object 
 10  Seats              5977 non-null   float64
 11  New_Price          824 non-null    object 
 12  Price              6019 non-null   float64
dtypes: float64(2), int64(2), object(9)
memory usage: 658.3+ KB


Quello che risalta subito all'occhio sono i missing values (NaN) in più di qualche features, dovremmo tenerne conto  

**Dizionario dei dati**  
Name: marchio + modello. E' qualitativa, e si potrebbe estrarre la marca.  
Location: altra variabile categorica, che descrive la città o la regione in cui l'auto è in vendita.   
Year: l'anno di produzione del modello. Numerica, discreta, scala a intervalli. Si potrebbe derivare l'età dell'auto.  
Kilometers_Driven: i chilometri totali percorsi dall'auto. Numerica, continua, scala ratio. Unità di misura: km.    
Fuel_type: tipo carburante. Categorica.  
Transmission: categorica binaria. Descrizione: automatica o manuale.  
Owner_Type: categorica ed ordinale perché "Firsthand" ha un valore maggiore.   
Mileage: indica il consumo di carburante, è numerica e continua, scala ratio, anche se in realtà non può consumare 0. Unità di misura: kmpl, km/kg.  
Engine: cilindrata del motore. E' numerica e continua, scala ratio. Unità di misura: cc.   
Power: la potenza massima del motore in bhp, è numerica e continua, scala ratio. Unità di misura: bhp.  
Seats: numero di posti a sedere, è numerica e discreta, ratio. Potrebbe definire la categoria del veicolo (SUV, monovolume ecc...)  
New_Price: il prezzo dell'auto nuova dello stesso modello. Numerica, continua, ratio. Valuta: INR Lakhs.  
Price: variabile target, è il prezzo di vendita dell'auto usata. Numerica, continua, ratio. Essendo una variabile continua, il problema da risolvere è un problema di regressione. Valuta: INR Lakhs.  

**Definizione degli obiettivi dell’analisi dei dati**  
Quali sono i fattori principali che determinano il prezzo (Price) di un'auto usata?  
In che modo l'età dell'auto (derivata da Year) e i chilometri percorsi (Kilometers_Driven) interagiscono nel definire il valore?  
Quale impatto hanno le caratteristiche tecniche come il tipo di carburante (Fuel_Type) e la Transmission sul prezzo?    
Esistono marche (estratte da Name) o località (Location) specifiche in cui le auto mantengono meglio il loro valore?  


**Pulizia e Preprocessing**  
Iniziamo capendo con quanti NaN abbiamo a che fare, e come gestirli.

In [140]:
df.isnull().sum()

Name                    0
Location                0
Year                    0
Kilometers_Driven       0
Fuel_Type               0
Transmission            0
Owner_Type              0
Mileage                 2
Engine                 36
Power                  36
Seats                  42
New_Price            5195
Price                   0
dtype: int64

In [141]:
for x in df:
    if df[x].isnull().sum() > 0:
        print(f"Percentuale di NaN in {x}: ", (df[x].isnull().sum()/df[x].size)*100, "%")


Percentuale di NaN in Mileage:  0.033228110981890685 %
Percentuale di NaN in Engine:  0.5981059976740323 %
Percentuale di NaN in Power:  0.5981059976740323 %
Percentuale di NaN in Seats:  0.6977903306197043 %
Percentuale di NaN in New_Price:  86.31001827546105 %


Prima di potere gestire questi valori mancanti dobbiamo sistemare un po' di cose. Intanto rimuoviamo le unità di misura e trasformiamo alcune colonne in numeriche.

La gestione della colonna Mileage è particolare, in quanto abbiamo due unità di misura differenti a seconda del carburante, naturalmente dobbiamo uniformare ad una sola unità di misura, e noi abbiamo scelto: km/kg.

In [142]:
df['Mileage'] = df['Mileage'].str.split(' ').str[0]
df['Mileage'] = pd.to_numeric(df['Mileage'], errors='coerce')

In [143]:
df['Power'] = df['Power'].str.split(' ').str[0]
df['Power'] = pd.to_numeric(df['Power'], errors='coerce')

In [144]:
df['New_Price'] = df['New_Price'].str.split(' ').str[0]
df['New_Price'] = pd.to_numeric(df['New_Price'], errors='coerce')

In [145]:
df['Engine'] = df['Engine'].str.split(' ').str[0]
df['Engine'] = pd.to_numeric(df['Engine'], errors='coerce')

In [146]:
df.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.60,998.0,58.16,5.0,NaN,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582.0,126.20,5.0,NaN,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.20,1199.0,88.70,5.0,8.61,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248.0,88.76,7.0,NaN,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968.0,140.80,5.0,NaN,17.74


In [147]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6019 entries, 0 to 6018
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               6019 non-null   object 
 1   Location           6019 non-null   object 
 2   Year               6019 non-null   int64  
 3   Kilometers_Driven  6019 non-null   int64  
 4   Fuel_Type          6019 non-null   object 
 5   Transmission       6019 non-null   object 
 6   Owner_Type         6019 non-null   object 
 7   Mileage            6017 non-null   float64
 8   Engine             5983 non-null   float64
 9   Power              5876 non-null   float64
 10  Seats              5977 non-null   float64
 11  New_Price          824 non-null    float64
 12  Price              6019 non-null   float64
dtypes: float64(6), int64(2), object(5)
memory usage: 658.3+ KB


Ora controlliamo i valori di ogni singola colonna, per capire se fanno parte del dominio. Ad esempio non può esserci un auto con 0 posti.

In [148]:
print(df["Location"].unique())

['Mumbai' 'Pune' 'Chennai' 'Coimbatore' 'Hyderabad' 'Jaipur' 'Kochi'
 'Kolkata' 'Delhi' 'Bangalore' 'Ahmedabad']


In [149]:
unique_values = df['Year'].unique()
bad_values = unique_values[(unique_values <= 0) | (unique_values >= 2025)]
print(bad_values)

[]


In [150]:
unique_values = df['Kilometers_Driven'].unique()
bad_values = unique_values[(unique_values <= 0)]
print(bad_values)

[]


In [151]:
print(df["Fuel_Type"].unique())

['CNG' 'Diesel' 'Petrol' 'LPG' 'Electric']


In [152]:
print(df["Transmission"].unique())

['Manual' 'Automatic']


In [153]:
print(df["Owner_Type"].unique())

['First' 'Second' 'Fourth & Above' 'Third']


In [154]:
unique_values = df['Mileage'].unique()
bad_values = unique_values[(unique_values <= 0)]
print(bad_values)

[0.]


In [155]:
bad_values = df.loc[df['Mileage'] <= 0, 'Mileage']
print(bad_values.value_counts())

Mileage
0.0    68
Name: count, dtype: int64


In [156]:
print(df['Engine'].unique())

[ 998. 1582. 1199. 1248. 1968.  814. 1461. 2755. 1598. 1462. 1497. 2179.
 2477. 1498. 2143. 1995. 1984. 1197. 2494. 1798. 2696. 2698. 1061. 1198.
 2987.  796.  624. 1999. 1991. 2694. 1120. 2498.  799. 2393. 1399. 1796.
 2148. 1396. 1950. 4806. 1998. 1086. 1193. 2982. 1493. 2967. 2993. 1196.
 1799. 2497. 2354. 1373. 2996. 1591. 2894. 5461. 1595.  936. 1997.   nan
 1896. 1390. 1364. 2199.  993.  999. 1405. 2956. 1794.  995. 2496. 1599.
 2400. 1495. 2523.  793. 4134. 1596. 1395. 2953. 1586. 2362. 1496. 1368.
 1298. 1956. 1299. 3498. 2835. 1150. 3198. 1343. 1499. 1186. 1590. 2609.
 2499. 2446. 1978. 2360. 3436. 2198. 4367. 2706. 1422. 2979. 1969. 1489.
 2489. 1242. 1388. 1172. 2495. 1194. 3200. 1781. 1341. 2773. 3597. 1985.
 2147. 1047. 2999. 2995. 2997. 1948. 2359. 4395. 2349. 2720. 1468. 3197.
 2487. 1597. 2771.   72. 4951.  970. 2925. 2200. 5000. 2149. 5998. 2092.
 5204. 2112. 1797.]


In [157]:
unique_values = df['Power'].unique()
bad_values = unique_values[(unique_values <= 0)]
print(bad_values)

[]


In [158]:
bad_values = df.loc[df['Seats'] <= 0, 'Seats']
print(bad_values.value_counts())

Seats
0.0    1
Name: count, dtype: int64


In [159]:
bad_values = df.loc[df['New_Price'] <= 0, 'New_Price']
print(bad_values.value_counts())

Series([], Name: count, dtype: int64)


In [160]:
bad_values = df.loc[df['Price'] <= 0, 'Price']
print(bad_values.value_counts())

Series([], Name: count, dtype: int64)


Notiamo che è tutto corretto tranne un elemento in Seats e 68 in Mileage. Potremmo semplicemente considerarli come dei NaN e gestirli come tali.

Ora possiamo gestire i NaN, abbiamo intenzione di imputare con la mediana, perché essendo un dataset con prezzi d'auto, potrebbe essere asimmetrica la distribuzione, quindi vogliamo optare per una scelta più robusta e sicura. (Mileage aspettare conversione di unità). Infine abbiamo deciso di droppare la colonna New_Price, in quanto ben l'86% dei dati è un valore NaN, e provare ad imputare creerebbe solo che rumore.

Percentuale di NaN in Mileage:  0.033228110981890685 %
Percentuale di NaN in Engine:  0.5981059976740323 %
Percentuale di NaN in Power:  0.5981059976740323 % 
Percentuale di NaN in Seats:  0.6977903306197043 % (0)
Percentuale di NaN in New_Price:  86.31001827546105 %



In [161]:
df = df.drop('New_Price', axis=1)

In [162]:
cols_to_impute = ['Power', 'Mileage', 'Engine', 'Seats']
df[cols_to_impute] = df[cols_to_impute].replace(0.0, np.nan)

for col in cols_to_impute:
    median_value = df[col].median()
    df[col] = df[col].fillna(median_value)

In [163]:
df['Engine'] = df['Engine'].astype('int64');
df['Seats'] = df['Seats'].astype('int64');

Abbiamo corretto gli ultimi Dtype. Ora ci siamo, il dataset è correttamente pulito.

La colonna name vorremmo fosse più rappresentativa, per questo motivo vorremo modificarle in due colonne una "Brand", e una "Model".

In [164]:
df['Brand'] = df['Name'].str.split(' ').str[0]
df['Model'] = df['Name'].str.split(' ').str[1]
df = df.drop('Name', axis=1)

In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6019 entries, 0 to 6018
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Location           6019 non-null   object 
 1   Year               6019 non-null   int64  
 2   Kilometers_Driven  6019 non-null   int64  
 3   Fuel_Type          6019 non-null   object 
 4   Transmission       6019 non-null   object 
 5   Owner_Type         6019 non-null   object 
 6   Mileage            6019 non-null   float64
 7   Engine             6019 non-null   int64  
 8   Power              6019 non-null   float64
 9   Seats              6019 non-null   int64  
 10  Price              6019 non-null   float64
 11  Brand              6019 non-null   object 
 12  Model              6019 non-null   object 
dtypes: float64(3), int64(4), object(6)
memory usage: 658.3+ KB


In [166]:
df.head()

,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price,Brand,Model
0,Mumbai,2010,72000,CNG,Manual,First,26.60,998,58.16,5,1.75,Maruti,Wagon
1,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.20,5,12.50,Hyundai,Creta
2,Chennai,2011,46000,Petrol,Manual,First,18.20,1199,88.70,5,4.50,Honda,Jazz
3,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7,6.00,Maruti,Ertiga
4,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968,140.80,5,17.74,Audi,A4


Verifichiamo che non ci siano errori di battitura o altro

In [167]:
df['Brand'].value_counts()

Brand
Maruti           1211
Hyundai          1107
Honda             608
Toyota            411
Mercedes-Benz     318
Volkswagen        315
Ford              300
Mahindra          272
BMW               267
Audi              236
Tata              186
Skoda             173
Renault           145
Chevrolet         121
Nissan             91
Land               60
Jaguar             40
Fiat               28
Mitsubishi         27
Mini               26
Volvo              21
Porsche            18
Jeep               15
Datsun             13
Force               3
ISUZU               2
Smart               1
Isuzu               1
Ambassador          1
Bentley             1
Lamborghini         1
Name: count, dtype: int64

In [168]:
df['Model'].unique()

array(['Wagon', 'Creta', 'Jazz', 'Ertiga', 'A4', 'EON', 'Micra', 'Innova',
       'Vento', 'Indica', 'Ciaz', 'City', 'Swift', 'Rover', 'Pajero',
       'Amaze', 'Duster', 'New', '3', 'S', 'A6', 'i20', 'Alto', 'WRV',
       'Corolla', 'Ssangyong', 'Vitara', 'KUV', 'M-Class', 'Polo', 'Nano',
       'Elantra', 'Xcent', 'Thar', 'Grand', 'KWID', 'i10', 'X-Trail',
       'Zen', 'Figo', 'C-Class', 'Cayenne', 'XUV500', 'Terrano', 'Brio',
       'Fiesta', 'Santro', 'Zest', 'Ritz', '5', 'Fortuner', 'Ecosport',
       'Verna', 'GO', 'Omni', 'Etios', 'XF', 'Eeco', 'Civic', 'V40', 'B',
       'Scorpio', 'CR-V', 'SLC', '1', 'Beat', 'Rapid', 'RS5', 'Superb',
       'X5', 'GLC', 'Countryman', 'Optra', 'Lodgy', 'E-Class', 'Baleno',
       'Laura', 'NuvoSport', 'Fabia', 'Indigo', 'Q3', 'Octavia', 'A8',
       'Verito', 'Cooper', 'Santa', 'X1', 'Accent', 'Tucson', 'GLE',
       'A-Star', 'Grande', 'X3', 'EcoSport', 'Q7', 'Jetta', 'GLA',
       'Celerio', 'Sumo', 'Accord', '6', 'Manza', 'Spark', 'Clubman'

Perfetto. Ora rimuoviamo eventuali righe duplicate

In [169]:
duplicate_count = df.duplicated().sum()

print(f"Ci sono {duplicate_count} righe duplicate.")

Ci sono 2 righe duplicate.


In [170]:
df.drop_duplicates(inplace=True)

Adesso passiamo alla gestione degli outliers.